In [ ]:
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
 
      

# Define the encoder function.
def one_hot_encode(labels):
    n_labels = len(labels)
    n_unique_labels = len(np.unique(labels))
    one_hot_encode = np.zeros((n_labels, n_unique_labels))
    one_hot_encode[np.arange(n_labels), labels] = 1
    return one_hot_encode

# Reading the dataset

df = pd.read_csv('C:\\Users\\Nikalal\\Desktop\\python work\\Neural Final year\\All_norm_normalized.csv')
# print(len(df.columns))
X = df[df.columns[0:257]].values#features
y = df[df.columns[257]]#labels

#print(y)

encoder = LabelEncoder()
encoder.fit(y)
y = encoder.transform(y)
Y = one_hot_encode(y)


# Shuffle the dataset to mix up the rows.
X, Y = shuffle(X, Y, random_state=1)
 
# Convert the dataset into train and test part
train_x, test_x, train_y, test_y = train_test_split(X, Y, test_size=0.20, random_state=415)


# Define the important parameters and variable to work with the tensors
learning_rate = 0.1
training_epochs = 5000
cost_history = np.empty(shape=[1], dtype=float)
n_dim = X.shape[1]#shape of the features(no.of columns)
#print("n_dim", n_dim)
n_class = 5
#model_path = "C:\\Users\\Nikalal\\Desktop\\python work\\Neural Final year\\Data\\meta\\meta"
 
# Define the number of hidden layers and number of neurons for each layer
n_hidden_1 = 60
n_hidden_2 = 60
#n_hidden_3 = 10
#n_hidden_4 = 10

x = tf.placeholder(tf.float32, [None, n_dim])#none-any value
W = tf.Variable(tf.zeros([n_dim, n_class]))
b = tf.Variable(tf.zeros([n_class]))
y_ = tf.placeholder(tf.float32, [None, n_class])

# Define the model
def multilayer_perceptron(x, weights, biases):
 
    # Hidden layer with RELU activationsd
    layer_1 = tf.add(tf.matmul(x, weights['h1']), biases['b1'])
    layer_1 = tf.nn.sigmoid(layer_1)
    
    # Hidden layer with sigmoid activation
    layer_2 = tf.add(tf.matmul(layer_1, weights['h2']), biases['b2'])
    layer_2 = tf.nn.sigmoid(layer_2)
    '''
    # Hidden layer with sigmoid activation
    layer_3 = tf.add(tf.matmul(layer_2, weights['h3']), biases['b3'])
    layer_3 = tf.nn.sigmoid(layer_3)
 
    # Hidden layer with RELU activation
    layer_4 = tf.add(tf.matmul(layer_3, weights['h4']), biases['b4'])
    layer_4 = tf.nn.relu(layer_4)
    '''
    # Output layer with linear activation
    out_layer = tf.matmul(layer_2, weights['out']) + biases['out']
    return out_layer

# Define the weights and the biases for each layer
 
weights = {
    'h1': tf.Variable(tf.truncated_normal([n_dim, n_hidden_1])),
    'h2': tf.Variable(tf.truncated_normal([n_hidden_1, n_hidden_2])),
    #'h3': tf.Variable(tf.truncated_normal([n_hidden_2, n_hidden_3])),
    #'h4': tf.Variable(tf.truncated_normal([n_hidden_3, n_hidden_4])),
    'out': tf.Variable(tf.truncated_normal([n_hidden_2, n_class]))
}
biases = {
    'b1': tf.Variable(tf.truncated_normal([n_hidden_1])),
    'b2': tf.Variable(tf.truncated_normal([n_hidden_2])),
    #'b3': tf.Variable(tf.truncated_normal([n_hidden_3])),
    #'b4': tf.Variable(tf.truncated_normal([n_hidden_4])),
    'out': tf.Variable(tf.truncated_normal([n_class]))
}
 
# Initialize all the variables

init = tf.global_variables_initializer()
 
saver = tf.train.Saver()
 
# Call your model defined
y = multilayer_perceptron(x, weights, biases)

# Define the cost function and optimizer
cost_function = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=y, labels=y_),name="cost")#Define loss function
training_step = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost_function)#learning algorithm
#training_step = tf.train.AdagradOptimizer(learning_rate,initial_accumulator_value=0.1,use_locking=False).minimize(cost_function)#learning algorithm

sess = tf.Session()
sess.run(init)

# Calculate the cost and the accuracy for each epoch
 
mse_history = []
accuracy_history = []
 
for epoch in range(training_epochs):
    sess.run(training_step, feed_dict={x: train_x, y_: train_y})#
    cost = sess.run(cost_function, feed_dict={x: train_x, y_: train_y})#calcukate cost
    cost_history = np.append(cost_history, cost)#append cost
    correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))#difference between actual and model output
    #tf.argmax Returns the index with the largest value across axes of a tensor
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))#Compute the mean of the actual and model output
    #print("Accuracy: ", (sess.run(accuracy, feed_dict={x: test_x, y_: test_y})))
    pred_y = sess.run(y, feed_dict={x: test_x})
    mse = tf.reduce_mean(tf.square(pred_y - test_y))#Computes the mean of square difference of predict and test sets
    mse_ = sess.run(mse)
    mse_history.append(mse_)
    
    accuracy = (sess.run(accuracy, feed_dict={x: train_x, y_: train_y}))
    #tf.summary.scalar("accuracy", accuracy)
    accuracy_history.append(accuracy)
    
 
    print('epoch : ', epoch, ' - ', 'cost: ', cost, " - MSE: ", mse_, "- Train Accuracy: ", accuracy)

####
save_path = saver.save(sess, model_path)
print("Model saved in file: %s" % save_path)
 
#Plot Accuracy Graph
plt.plot(accuracy_history)
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.show()

# Print the final accuracy
correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print("Test Accuracy: ", (sess.run(accuracy, feed_dict={x: test_x, y_: test_y})))
 
# Print the final mean square error

pred_y = sess.run(y, feed_dict={x: test_x})
mse = tf.reduce_mean(tf.square(pred_y - test_y))
print("MSE: %.4f" % sess.run(mse))
